Using Keras, SELU multilayer perceptrons and Hyperopt on Tox21
====


In [91]:
import numpy as np
import pandas as pd
from scipy import io
    
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, concatenate
from keras.layers.noise import AlphaDropout
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
import sklearn.utils 

import tempfile

import tensorflow as tf

In [1]:
source_directory = "/media/matthias/Big-disk/tox21-hochreiter/"

# load dense features and labels
y_tr = pd.read_csv(source_directory + 'tox21_labels_train.csv.gz', index_col=0, compression="gzip")
y_te = pd.read_csv(source_directory + 'tox21_labels_test.csv.gz', index_col=0, compression="gzip")
x_tr_dense = pd.read_csv(source_directory + 'tox21_dense_train.csv.gz', index_col=0, compression="gzip").values
x_te_dense = pd.read_csv(source_directory + 'tox21_dense_test.csv.gz', index_col=0, compression="gzip").values

# load sparse features
x_tr_sparse = io.mmread(source_directory + 'tox21_sparse_train.mtx.gz').tocsc()
x_te_sparse = io.mmread(source_directory + 'tox21_sparse_test.mtx.gz').tocsc()

# filter out very sparse features. combine filtered sparse features with dense features
sparse_col_idx = ((x_tr_sparse > 0).mean(0) > 0.05).A.ravel()
x_tr = np.hstack([x_tr_dense, x_tr_sparse[:, sparse_col_idx].A])
x_te = np.hstack([x_te_dense, x_te_sparse[:, sparse_col_idx].A])

# scale input features to zero mean and unit variance (might be important for SELU network)
scaler = preprocessing.StandardScaler().fit(x_tr)
x_tr = scaler.transform(x_tr)
x_te = scaler.transform(x_te)

Using TensorFlow backend.


In [2]:
# All target assays: ['NR.AhR', 'NR.AR', 'NR.AR.LBD', 'NR.Aromatase', 'NR.ER', 'NR.ER.LBD',
#       'NR.PPAR.gamma', 'SR.ARE', 'SR.ATAD5', 'SR.HSE', 'SR.MMP', 'SR.p53']

def data(target):
    """
    Returns data for a specific assays (i.e., samples where result for assay is available)
    """
    
    # filter out data where no results for target assay are available    
    rows_tr = np.isfinite(y_tr[target]).values
    rows_te = np.isfinite(y_te[target]).values
    
    x_train = x_tr[rows_tr] 
    y_train = y_tr[target][rows_tr]
    x_test = x_te[rows_te]
    y_test = y_te[target][rows_te]   
    
    return x_train, y_train, x_test, y_test


In [124]:
def f_nn(p):
    # add constant parameters to parameter dictionary (workaround for hyperopt, take care to set global variable!)
    p = {**p, **constant_parameters}
    
    print("\n")
    print(p)
    
    try:
        max_epochs = 1000

        # some parameters need to be cast to integer
        p['hidden layers'] = int(p['hidden layers'])
        p['hidden units'] = int(p['hidden units'])
        p['batch size'] = int(p['batch size'])

        number_of_features = x_train.shape[1]
        
        # clear session (to avoid clutter from old models)
        K.clear_session()
        
        # the model is a classical multilayer perceptron with SELU activations and Alpha dropout
        inputs = Input(shape=(number_of_features,))
        nn = Dense(p['hidden units'], activation='selu', kernel_initializer=p['kernel initializer'])(inputs)
                
        for i in range(p['hidden layers']):
            if p['shape'] == 'triangle':
                # unit decrement per layer if triangle shape is selected, last hidden layer has 4 units
                difference = max(p['hidden units'] - 4, 4)
                decrement_per_layer = difference / p['hidden layers']
                number_of_units_in_this_layer = int(p['hidden units'] - (decrement_per_layer * i))
            else:
                number_of_units_in_this_layer = p['hidden units']
            nn = AlphaDropout(p['dropout rate'])(nn)
            nn = Dense(p['hidden units'], activation='selu', kernel_initializer=p['kernel initializer'])(nn)
        
        if p['skip_connection']:
            # add additional layer before output layer, feed in original input
            nn = concatenate([nn, inputs])
            nn = Dense(p['hidden units'], activation='selu', kernel_initializer=p['kernel initializer'])(nn)
        
        outputs = Dense(1, activation='sigmoid')(nn)

        if p['optimizer'] == 'sgd':
            optimizer=optimizers.sgd(lr=p['learning rate'], momentum=0.1, nesterov=True)
        elif p['optimizer'] == 'adam':
            optimizer=optimizers.adam(lr=p['learning rate'])
        elif p['optimizer'] == 'RMSprop':
            optimizer=optimizers.RMSprop(lr=p['learning rate'])
        elif p['optimizer'] == 'Adadelta':
            optimizer=optimizers.RMSprop(lr=p['learning rate'])

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])

        # weigh classes (in case labels are unbalanced)
        class_weight = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

        _, tmpfn = tempfile.mkstemp()

        callbacks = [EarlyStopping(patience=p['patience']), 
                     ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, min_lr=1e-06, verbose=1),
                     ModelCheckpoint(tmpfn, save_best_only=True, save_weights_only=True)]
        model.fit(x_train, y_train, epochs=max_epochs, batch_size=p['batch size'], validation_data=(x_test, y_test), 
              callbacks=callbacks, class_weight=class_weight, verbose=p['verbose'])

        # load model weights of epoch that had best results
        model.load_weights(tmpfn)

        p_te = model.predict(x_test)

        # add column for other class, adding up to probability of 1
        new_col = np.subtract(1,p_te).reshape((p_te.shape[0],1))
        p_te_both_classes = np.append(new_col,p_te, 1)

        auc_te = roc_auc_score(y_test, p_te_both_classes[:, 1])
        print("%15s score on test: %3.5f" % (target, auc_te))

        return {'loss': -auc_te, 'status': STATUS_OK, 'model': model}
    
    except tf.errors.ResourceExhaustedError: 
        # fail gracefully if model is too large to fit into memory (so hyperparameter search can continue)
        print("ResourceExhaustedError occurred, skipping...")
        return {'status': STATUS_FAIL}

Train and evaluate model with best hyperparameters
----------

In [14]:
from hyperopt import space_eval 

for target in ['NR.AhR', 'NR.AR', 'NR.AR.LBD', 'NR.Aromatase', 'NR.ER', 'NR.ER.LBD', 
               'NR.PPAR.gamma', 'SR.ARE', 'SR.ATAD5', 'SR.HSE', 'SR.MMP', 'SR.p53']:
    
    x_train, y_train, x_test, y_test = data(target)

    space = {
        'batch size' : hp.quniform('batch size', 10, 100, 1),
        'hidden layers': hp.quniform('hidden layers', 0, 16, 1),
        'hidden units': hp.quniform('hidden units', 100, 1000, 1),
        'dropout rate': hp.uniform('dropout rate', .01, .1),
        'learning rate': hp.uniform('learning rate', 1e-07, 1e-05 )}

    constant_parameters = {
        'shape': 'square',
        'optimizer': 'adam',
        'kernel initializer': 'lecun_normal',
        'patience': 30
    }

    trials = Trials()
    best = fmin(f_nn, space, algo=tpe.suggest, max_evals=25, trials=trials)

    print('======== best for ' + target + ': ')
    print(space_eval(space, best))



{'batch size': 22.0, 'dropout rate': 0.07136936311571818, 'hidden layers': 7.0, 'hidden units': 268.0, 'learning rate': 9.242742375853505e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.89799


{'batch size': 82.0, 'dropout rate': 0.014077402974367395, 'hidden layers': 2.0, 'hidden units': 392.0, 'learning rate': 6.67985064253016e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.89161


{'batch size': 42.0, 'dropout rate': 0.09251178661959933, 'hidden layers': 7.0, 'hidden units': 909.0, 'learning rate': 5.355445692499389e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.5657

 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.89033


{'batch size': 25.0, 'dropout rate': 0.03106785170629097, 'hidden layers': 13.0, 'hidden units': 461.0, 'learning rate': 6.338565998807057e-07, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
 32/610 [>.............................] - ETA: 1s         NR.AhR score on test: 0.86016


{'batch size': 29.0, 'dropout rate': 0.06503501904889716, 'hidden layers': 9.0, 'hidden units': 594.0, 'learning rate': 4.928405421328021e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.56575067  4.30224261]
 32/610 [>.............................] - ETA: 1s         NR.AhR score on test: 0.86511
======== best for NR.AhR: 
{'batch size': 27.0, 'dropout rate': 0.037447677802347416, 'hidden layers': 10.0, 'hidden units': 407.0, 'learning rate': 4.27275024188765e-06}



Class weights:
[  0.52065581  12.60313316]
 32/586 [>.............................] - ETA: 1s          NR.AR score on test: 0.76800


{'batch size': 12.0, 'dropout rate': 0.0322678967833666, 'hidden layers': 16.0, 'hidden units': 103.0, 'learning rate': 6.506330961847355e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52065581  12.60313316]
 32/586 [>.............................] - ETA: 1s          NR.AR score on test: 0.80328


{'batch size': 11.0, 'dropout rate': 0.038616255221048224, 'hidden layers': 14.0, 'hidden units': 207.0, 'learning rate': 6.428815730482679e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52065581  12.60313316]
 32/586 [>.............................] - ETA: 1s          NR.AR score on test: 0.75929


{'batch size': 19.0, 'dropout rate': 0.035114738391035724, 'hidden layers': 14.0, 'hidden units': 109.0, 'learning rate': 

Class weights:
[  0.51796372  14.41693811]
 32/582 [>.............................] - ETA: 1s      NR.AR.LBD score on test: 0.49260


{'batch size': 39.0, 'dropout rate': 0.028099177036559966, 'hidden layers': 16.0, 'hidden units': 303.0, 'learning rate': 6.997936744715498e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.51796372  14.41693811]
 32/582 [>.............................] - ETA: 1s      NR.AR.LBD score on test: 0.55684


{'batch size': 93.0, 'dropout rate': 0.023109411761090885, 'hidden layers': 2.0, 'hidden units': 757.0, 'learning rate': 1.994323226570051e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.51796372  14.41693811]
 32/582 [>.............................] - ETA: 0s      NR.AR.LBD score on test: 0.76307


{'batch size': 75.0, 'dropout rate': 0.013374427869563022, 'hidden layers': 13.0, 'hidden units': 458.0, 'learning rate':

Class weights:
[ 0.52676296  9.84126984]
 32/528 [>.............................] - ETA: 1s   NR.Aromatase score on test: 0.69965


{'batch size': 36.0, 'dropout rate': 0.06552400634466429, 'hidden layers': 0.0, 'hidden units': 648.0, 'learning rate': 1.7867758860417195e-07, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.52676296  9.84126984]
 32/528 [>.............................] - ETA: 0s   NR.Aromatase score on test: 0.79073


{'batch size': 80.0, 'dropout rate': 0.022171283210297765, 'hidden layers': 16.0, 'hidden units': 263.0, 'learning rate': 1.953967863667563e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.52676296  9.84126984]
 32/528 [>.............................] - ETA: 1s   NR.Aromatase score on test: 0.73127


{'batch size': 75.0, 'dropout rate': 0.09005642766930387, 'hidden layers': 2.0, 'hidden units': 286.0, 'learning rate': 3.75073

...

 32/555 [>.............................] - ETA: 0s         SR.ARE score on test: 0.79507
======== best for SR.ARE: 
{'batch size': 42.0, 'dropout rate': 0.07035786455532222, 'hidden layers': 1.0, 'hidden units': 197.0, 'learning rate': 9.392678443347974e-06}


{'batch size': 26.0, 'dropout rate': 0.017059349876564302, 'hidden layers': 1.0, 'hidden units': 474.0, 'learning rate': 5.900253675506423e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52016667  12.89669421]
 32/622 [>.............................] - ETA: 0s       SR.ATAD5 score on test: 0.78821


{'batch size': 95.0, 'dropout rate': 0.07355255275435046, 'hidden layers': 2.0, 'hidden units': 852.0, 'learning rate': 3.6122381597964968e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52016667  12.89669421]
 32/622 [>.............................] - ETA: 0s       SR.ATAD5 score on test: 0.707

Class weights:
[  0.52016667  12.89669421]
 32/622 [>.............................] - ETA: 1s       SR.ATAD5 score on test: 0.68696


{'batch size': 38.0, 'dropout rate': 0.09977000941695163, 'hidden layers': 9.0, 'hidden units': 824.0, 'learning rate': 7.066825515044912e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52016667  12.89669421]
 32/622 [>.............................] - ETA: 0s       SR.ATAD5 score on test: 0.74288


{'batch size': 18.0, 'dropout rate': 0.054055382403718655, 'hidden layers': 14.0, 'hidden units': 374.0, 'learning rate': 4.312985518707168e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[  0.52016667  12.89669421]
 32/622 [>.............................] - ETA: 1s       SR.ATAD5 score on test: 0.82678


{'batch size': 63.0, 'dropout rate': 0.05174275428841806, 'hidden layers': 14.0, 'hidden units': 377.0, 'learning rate': 4

Class weights:
[ 0.52744705  9.60844749]
 32/610 [>.............................] - ETA: 1s         SR.HSE score on test: 0.79105


{'batch size': 62.0, 'dropout rate': 0.08048115860238501, 'hidden layers': 5.0, 'hidden units': 956.0, 'learning rate': 6.328540612978295e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.52744705  9.60844749]
 32/610 [>.............................] - ETA: 0s         SR.HSE score on test: 0.82692


{'batch size': 78.0, 'dropout rate': 0.042249533265904415, 'hidden layers': 1.0, 'hidden units': 607.0, 'learning rate': 6.924519848071295e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.52744705  9.60844749]
 32/610 [>.............................] - ETA: 0s         SR.HSE score on test: 0.85080


{'batch size': 78.0, 'dropout rate': 0.040421274398621414, 'hidden layers': 1.0, 'hidden units': 611.0, 'learning rate': 6.909218

Class weights:
[ 0.59250549  3.20254237]
 32/543 [>.............................] - ETA: 1s         SR.MMP score on test: 0.92039


{'batch size': 46.0, 'dropout rate': 0.075362197253205, 'hidden layers': 15.0, 'hidden units': 516.0, 'learning rate': 1.2218986007862182e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.59250549  3.20254237]
 32/543 [>.............................] - ETA: 1s         SR.MMP score on test: 0.67333


{'batch size': 58.0, 'dropout rate': 0.053522242402782964, 'hidden layers': 14.0, 'hidden units': 746.0, 'learning rate': 9.535339237161361e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.59250549  3.20254237]
 32/543 [>.............................] - ETA: 1s         SR.MMP score on test: 0.90769


{'batch size': 91.0, 'dropout rate': 0.03918473049609136, 'hidden layers': 13.0, 'hidden units': 605.0, 'learning rate': 2.22818

 32/616 [>.............................] - ETA: 0s         SR.p53 score on test: 0.77888


{'batch size': 95.0, 'dropout rate': 0.029162052921794843, 'hidden layers': 10.0, 'hidden units': 518.0, 'learning rate': 7.255870737968997e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.53388103  7.87876106]
 32/616 [>.............................] - ETA: 1s         SR.p53 score on test: 0.73650


{'batch size': 62.0, 'dropout rate': 0.09163087648880294, 'hidden layers': 7.0, 'hidden units': 164.0, 'learning rate': 6.891298693882184e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30}
Class weights:
[ 0.53388103  7.87876106]
 32/616 [>.............................] - ETA: 0s         SR.p53 score on test: 0.75169


{'batch size': 92.0, 'dropout rate': 0.09929882808843414, 'hidden layers': 0.0, 'hidden units': 165.0, 'learning rate': 8.976590425443003e-06, 'shape': 'square', 'optimi

In [128]:
best_params = {'NR.AhR':
                   {'batch size': 27.0,
                    'dropout rate': 0.037447677802347416,
                    'hidden layers': 10.0,
                    'hidden units': 407.0,
                    'learning rate': 4.27275024188765e-06},
               'NR.AR':
                   {'batch size': 26.0, 'dropout rate': 0.03876138988120934, 'hidden layers': 14.0,
                    'hidden units': 170.0, 'learning rate': 9.970028110390661e-06},

               'NR.AR.LBD':
                   {'batch size': 75.0, 'dropout rate': 0.019588919402218484, 'hidden layers': 1.0,
                    'hidden units': 511.0, 'learning rate': 8.97807172349493e-06},

               'NR.Aromatase':
                   {'batch size': 62.0, 'dropout rate': 0.015271491578662429, 'hidden layers': 1.0,
                    'hidden units': 646.0, 'learning rate': 6.192920190335587e-06},

               'NR.ER':
                   {'batch size': 99.0, 'dropout rate': 0.04032642692556794, 'hidden layers': 2.0,
                    'hidden units': 393.0, 'learning rate': 9.434501332155454e-06},

               'NR.ER.LBD':
                   {'batch size': 51.0, 'dropout rate': 0.09587500147505609, 'hidden layers': 1.0,
                    'hidden units': 629.0, 'learning rate': 1.5033620458229143e-07},

               'NR.PPAR.gamma':
                   {'batch size': 65.0, 'dropout rate': 0.027669664203144387, 'hidden layers': 8.0,
                    'hidden units': 1000.0, 'learning rate': 8.77272816674485e-06},

               'SR.ARE':
                   {'batch size': 42.0, 'dropout rate': 0.07035786455532222, 'hidden layers': 1.0,
                    'hidden units': 197.0, 'learning rate': 9.392678443347974e-06},

               'SR.ATAD5':
                   {'batch size': 18.0, 'dropout rate': 0.054055382403718655, 'hidden layers': 14.0,
                    'hidden units': 374.0, 'learning rate': 4.312985518707168e-06},

               'SR.HSE':
                   {'batch size': 67.0, 'dropout rate': 0.05719563006214853, 'hidden layers': 1.0,
                    'hidden units': 842.0, 'learning rate': 6.411213666905385e-06},

               'SR.MMP':
                   {'batch size': 30.0, 'dropout rate': 0.025349463629542116, 'hidden layers': 1.0,
                    'hidden units': 739.0, 'learning rate': 5.574733935242791e-06},

               'SR.p53':
                   {'batch size': 39.0, 'dropout rate': 0.014054479303884149, 'hidden layers': 0.0,
                    'hidden units': 372.0, 'learning rate': 9.598321722798496e-06}}

constant_parameters = {
        'shape': 'square',
        'optimizer': 'adam',
        'kernel initializer': 'lecun_normal',
        'patience': 30,
        'verbose': 0,
        'skip_connection': False
    }

scores = np.array([])

for target in ['NR.AhR', 'NR.AR', 'NR.AR.LBD', 'NR.Aromatase', 'NR.ER', 'NR.ER.LBD', 
               'NR.PPAR.gamma', 'SR.ARE', 'SR.ATAD5', 'SR.HSE', 'SR.MMP', 'SR.p53']:
    target = target
    x_train, y_train, x_test, y_test = data(target)
    result = f_nn(best_params[target])
    scores = np.append(scores, -result['loss'])

np.average(scores)



{'batch size': 27.0, 'dropout rate': 0.037447677802347416, 'hidden layers': 10.0, 'hidden units': 407.0, 'learning rate': 4.27275024188765e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30, 'verbose': 0, 'skip_connection': False}
         NR.AhR score on test: 0.86424


{'batch size': 26.0, 'dropout rate': 0.03876138988120934, 'hidden layers': 14.0, 'hidden units': 170.0, 'learning rate': 9.970028110390661e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30, 'verbose': 0, 'skip_connection': False}
          NR.AR score on test: 0.79878


{'batch size': 75.0, 'dropout rate': 0.019588919402218484, 'hidden layers': 1.0, 'hidden units': 511.0, 'learning rate': 8.97807172349493e-06, 'shape': 'square', 'optimizer': 'adam', 'kernel initializer': 'lecun_normal', 'patience': 30, 'verbose': 0, 'skip_connection': False}
      NR.AR.LBD score on test: 0.74630


{'batch size': 62.0, 'dropout rate': 0.015

0.80440055678918998

In [129]:
constant_parameters = {
        'shape': 'square',
        'optimizer': 'sgd',
        'kernel initializer': 'lecun_normal',
        'patience': 30,
        'verbose': 0
}

parameters = {
    'batch size': 100.0, 
    'dropout rate': 0.03, 
    'hidden layers': 11.0, 
    'hidden units': 2000.0, 
    'learning rate': 8e-03,
    'skip_connection': False
}

scores = np.array([])

for target in ['NR.AhR', 'NR.AR', 'NR.AR.LBD', 'NR.Aromatase', 'NR.ER', 'NR.ER.LBD', 
               'NR.PPAR.gamma', 'SR.ARE', 'SR.ATAD5', 'SR.HSE', 'SR.MMP', 'SR.p53']:
    target = target
    x_train, y_train, x_test, y_test = data(target)
    result = f_nn(parameters)
    scores = np.append(scores, -result['loss'])

np.average(scores)




{'batch size': 100.0, 'dropout rate': 0.03, 'hidden layers': 11.0, 'hidden units': 2000.0, 'learning rate': 0.008, 'skip_connection': False, 'shape': 'square', 'optimizer': 'sgd', 'kernel initializer': 'lecun_normal', 'patience': 30, 'verbose': 0}
         NR.AhR score on test: 0.88370


{'batch size': 100.0, 'dropout rate': 0.03, 'hidden layers': 11.0, 'hidden units': 2000.0, 'learning rate': 0.008, 'skip_connection': False, 'shape': 'square', 'optimizer': 'sgd', 'kernel initializer': 'lecun_normal', 'patience': 30, 'verbose': 0}
          NR.AR score on test: 0.79428


{'batch size': 100.0, 'dropout rate': 0.03, 'hidden layers': 11.0, 'hidden units': 2000.0, 'learning rate': 0.008, 'skip_connection': False, 'shape': 'square', 'optimizer': 'sgd', 'kernel initializer': 'lecun_normal', 'patience': 30, 'verbose': 0}
      NR.AR.LBD score on test: 0.73345


{'batch size': 100.0, 'dropout rate': 0.03, 'hidden layers': 11.0, 'hidden units': 2000.0, 'learning rate': 0.008, 'skip_connection

0.79990110075079157